In [2]:
from pyspark.sql.functions import mean, min, max
import pandas as pd
import csv

In [2]:
df_sleep = spark.read.json('/home/hadoop01/sokulee/*/*_sleep.json')

KeyboardInterrupt: 

In [4]:
df_sleep.printSchema()

root
 |-- sleep: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- awakeCount: long (nullable = true)
 |    |    |-- awakeDuration: long (nullable = true)
 |    |    |-- awakeningsCount: long (nullable = true)
 |    |    |-- dateOfSleep: string (nullable = true)
 |    |    |-- duration: long (nullable = true)
 |    |    |-- efficiency: long (nullable = true)
 |    |    |-- isMainSleep: boolean (nullable = true)
 |    |    |-- logId: long (nullable = true)
 |    |    |-- minuteData: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- dateTime: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- minutesAfterWakeup: long (nullable = true)
 |    |    |-- minutesAsleep: long (nullable = true)
 |    |    |-- minutesAwake: long (nullable = true)
 |    |    |-- minutesToFallAsleep: long (nullable = true)
 |    |    |-- restlessCount: long (nullable = true)
 |

In [5]:
df_sleep.select(mean(df_sleep['summary']["totalMinutesAsleep"]), min(df_sleep['summary']["totalMinutesAsleep"]), max(df_sleep['summary']["totalMinutesAsleep"])).show()

+----------------------------------+----------------------------------+----------------------------------+
|avg(summary['totalMinutesAsleep'])|min(summary['totalMinutesAsleep'])|max(summary['totalMinutesAsleep'])|
+----------------------------------+----------------------------------+----------------------------------+
|                306.74880239520957|                                 0|                              1199|
+----------------------------------+----------------------------------+----------------------------------+



# 데이터 가져오기 그리고 하루 평균으로 구하기

In [2]:
steps = []
user = []
sleep = []
hearts = []

In [3]:
dirs = []
for file in os.listdir("/home/hadoop01/sokulee"):
    if file != '.DS_Store':
        dirs.append("sokulee/" + file)
        user.append(file)

In [9]:
user


['A049',
 'A096',
 'A068',
 'A040',
 'A050',
 'A098',
 'A020',
 'A07',
 'A010',
 'A058',
 'A053',
 'A083',
 'A05',
 'A041',
 'A034',
 'A048',
 'A017',
 'A080',
 'A08',
 'A01',
 'A026',
 'A021',
 'A037',
 'A06',
 'A063',
 'A061',
 'A036',
 'A094',
 'A039',
 'A062',
 'A043',
 'A060',
 'A029',
 'A092',
 'A025',
 'A073',
 'A018',
 'A04',
 'A019',
 'A024',
 'A031',
 'A065',
 'A054',
 'A093',
 'A045',
 'A081',
 'A016',
 'A032',
 'A02',
 'A03',
 'A035',
 'A056',
 'A028',
 'A038',
 'A059',
 'A030',
 'A033',
 'A044',
 'A022',
 'A052',
 'A084',
 'A057',
 'A072',
 'A042',
 'A071',
 'A027',
 'A097',
 'A047']

In [4]:
for dir in dirs:
    user_steps = spark.read.json(dir + "/*_steps.json" )
    user_sleep = spark.read.json(dir + "/*_sleep.json" )
    user_heart = spark.read.json(dir + "/*_heart.json" )
    
    print(dir)
    
    step_mean = user_steps.select(mean(user_steps['activities-steps'][0]['value']))
    sleep_mean = user_sleep.select(mean(user_sleep['summary']['totalMinutesAsleep']))
    heart_mean = user_heart.select(mean(user_heart['activities-heart'][0]['value']['heartRateZones'][0]['minutes'] +
                                        user_heart['activities-heart'][0]['value']['heartRateZones'][1]['minutes'] +
                                        user_heart['activities-heart'][0]['value']['heartRateZones'][2]['minutes'] + 
                                        user_heart['activities-heart'][0]['value']['heartRateZones'][3]['minutes']))
    
    steps.append(step_mean.head()[0])
    sleep.append(sleep_mean.head()[0])
    hearts.append(heart_mean.head()[0])
    

sokulee/A049
sokulee/A096
sokulee/A068
sokulee/A040
sokulee/A050
sokulee/A098
sokulee/A020
sokulee/A07
sokulee/A010
sokulee/A058
sokulee/A053
sokulee/A083
sokulee/A05
sokulee/A041
sokulee/A034
sokulee/A048
sokulee/A017
sokulee/A080
sokulee/A08
sokulee/A01
sokulee/A026
sokulee/A021
sokulee/A037
sokulee/A06
sokulee/A063
sokulee/A061
sokulee/A036
sokulee/A094
sokulee/A039
sokulee/A062
sokulee/A043
sokulee/A060
sokulee/A029
sokulee/A092
sokulee/A025
sokulee/A073
sokulee/A018
sokulee/A04
sokulee/A019
sokulee/A024
sokulee/A031
sokulee/A065
sokulee/A054
sokulee/A093
sokulee/A045
sokulee/A081
sokulee/A016
sokulee/A032
sokulee/A02
sokulee/A03
sokulee/A035
sokulee/A056
sokulee/A028
sokulee/A038
sokulee/A059
sokulee/A030
sokulee/A033
sokulee/A044
sokulee/A022
sokulee/A052
sokulee/A084
sokulee/A057
sokulee/A072
sokulee/A042
sokulee/A071
sokulee/A027
sokulee/A097
sokulee/A047


# top step 과 top sleep 의 유클리디안으로 5명 구하기

In [22]:
data = pd.DataFrame({"user" : user , "steps" : steps , "sleep" : sleep}).round(2)

In [27]:
data.sort_values('steps', ascending=False, inplace=True)
top_step_data = data[:1]
top_step_data

,sleep,steps,user
34,271.82,21666.4,A025


In [28]:
data.sort_values('sleep', ascending=False, inplace=True)
top_sleep_data = data[:1]
top_sleep_data

,sleep,steps,user
65,471.37,9293.2,A027


In [30]:
import math

def euclidean_distance(a, b):    
    return math.sqrt( (a['sleep'] - b['sleep'])**2 + 
                      (a['steps'] - b['steps'])**2 )

def norm_euclidean_distance(a, b):
    return 1 / (euclidean_distance(a, b) + 1)

In [39]:
top5_step = []
top5_sleep = []

In [40]:
for index, row in data.iterrows():
    dist = norm_euclidean_distance(top_sleep_data, row)
    top5_sleep.append(dist)

In [41]:
for index, row in data.iterrows():
    dist = norm_euclidean_distance(top_step_data, row)
    top5_step.append(dist)

In [42]:
data['top5_step'] = top5_step  

In [44]:
data['top5_sleep'] = top5_sleep

In [48]:
data.sort_values('top5_step', ascending=False, inplace=True)
top_sleep_data = data[1:6]
top_sleep_data

,sleep,steps,user,top5_step,top5_sleep
20,338.86,21405.82,A026,0.003703,0.000083
30,360.76,19486.72,A043,0.000458,0.000098
44,372.76,16718.14,A045,0.000202,0.000135
29,213.18,16331.34,A062,0.000187,0.000142
42,342.38,16217.22,A054,0.000183,0.000144


In [49]:
data.sort_values('top5_sleep', ascending=False, inplace=True)
top_sleep_data = data[1:6]
top_sleep_data

,sleep,steps,user,top5_step,top5_sleep
41,471.37,9293.20,A065,0.000081,1.000000
62,421.60,9161.90,A072,0.000080,0.007071
55,253.49,9554.96,A030,0.000083,0.002928
60,249.44,8866.20,A084,0.000078,0.002074
58,426.96,8788.12,A022,0.000078,0.001968


# 끝

# k mean 클러스터링

In [ ]:
data = pd.DataFrame({"user" : user , "steps" : steps , "sleep" : sleep, "hearts" : hearts}).round(2)

In [24]:
k_data = pd.DataFrame({"steps" : steps , "sleep" : sleep, "hearts" : hearts}).round(2)

In [45]:
data

,sleep,steps,user,top5_step,top5_sleep
65,471.37,9293.20,A027,0.000081,1.000000
41,471.37,9293.20,A065,0.000081,1.000000
67,442.56,8274.74,A047,0.000075,0.000981
0,427.82,6516.66,A049,0.000066,0.000360
9,427.12,5413.86,A058,0.000062,0.000258
58,426.96,8788.12,A022,0.000078,0.001968
62,421.60,9161.90,A072,0.000080,0.007071
22,402.78,8267.06,A037,0.000075,0.000971
52,401.37,8428.42,A028,0.000076,0.001151
2,399.06,11698.64,A068,0.000100,0.000415


In [25]:
k_data['hearts'] = k_data['hearts'].map('1:{:.2f}'.format)
k_data['sleep'] = k_data['sleep'].map('2:{:.2f}'.format)
k_data['steps'] = k_data['steps'].map('3:{:.2f}'.format)
k_data

,hearts,sleep,steps
0,1:1402.24,2:427.82,3:6516.66
1,1:1042.32,2:209.86,3:10045.66
2,1:1371.14,2:399.06,3:11698.64
3,1:1139.06,2:260.44,3:14548.76
4,1:1065.27,2:171.56,3:7605.64
5,1:1053.75,2:13.56,3:5904.54
6,1:1157.08,2:295.27,3:13525.16
7,1:1383.88,2:398.51,3:11689.24
8,1:1166.52,2:393.27,3:5334.50
9,1:1235.49,2:427.12,3:5413.86


In [28]:
#k_mean를 하기위해 txt 파일로 저장하는 것
k_data.to_csv('avr.txt', sep=' ', header=False)

# k_mean 하기

In [18]:
from pyspark.ml.clustering import KMeans

In [19]:
dataset = spark.read.format("libsvm").load('avr.txt')

In [20]:
# 뭉쳐져있는 각각의 data 중에서 중심인 4개를 뽑아냄
#setK 의 값에 따라 중심 값의 수를 뽑아낸다.
# k means 하기
kmeans = KMeans().setK(4).setSeed(1)
model = kmeans.fit(dataset)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Within Set Sum of Squared Errors = 99560628.37097663
Cluster Centers: 
[ 1107.633   277.322  6225.785]
[  1291.73333333    323.81333333  20852.98      ]
[ 1203.31814815   314.28666667  9630.59851852]
[  1238.18611111    326.47611111  14611.09      ]


# 유클리디안 하기

In [32]:
me = pd.Series([900, 500, 5000, 'A000'], index = ['hearts' , 'sleep', 'steps', 'user'])

In [35]:
import math

def euclidean_distance(a, b):    
    return math.sqrt( (a['sleep'] - b['sleep'])**2 + 
                      (a['steps'] - b['steps'])**2 + 
                      (a['hearts'] - b['hearts'])**2  )

def norm_euclidean_distance(a, b):
    return 1 / (euclidean_distance(a, b) + 1)

In [36]:
normoalized_dist = -1
nearest_user = 'A00'
for index, row in data.iterrows():
    dist = norm_euclidean_distance(me, row)
    if normoalized_dist < dist:
        normoalized_dist = dist
        nearest_user = row['user']

print("Nearest user is %s, normalized_dist = %.10f" %(nearest_user, normoalized_dist))

Nearest user is A010, normalized_dist = 0.0022634106


# 내가 속한 클러스터는 무엇인가?!?!?

In [21]:
import numpy as np
me = np.array([900, 500, 5000])

In [22]:
import math

def euclidean_distance(a, b):    
    return math.sqrt( (a[0] - b[0])**2 + 
                      (a[1] - b[1])**2 + 
                      (a[2] - b[2])**2  )

def norm_euclidean_distance(a, b):
    return 1 / (euclidean_distance(a, b) + 1)

In [26]:
normoalized_dist = -1
nearest_center = []
for center in centers:
    dist = norm_euclidean_distance(me, center)
    if normoalized_dist < dist:
        normoalized_dist = dist
        nearest_center = center

print("my 클러스터 :", nearest_center)

my 클러스터 : [ 1107.633   277.322  6225.785]


# movie 유클리디안

In [9]:
movies = pd.read_csv('top_user.csv')
top1 = movies[:1]

In [15]:
import math
from scipy.spatial import distance


def norm_euclidean_distance(a, b):
    return 1/(distance.euclidean(a,b)+1)

In [6]:
top5_count = []

In [16]:
for index, row in movies.iterrows():
    dist = norm_euclidean_distance(top1, row)
    top5_count.append(dist)

In [17]:
top5_count[:20]

[1.0,
 0.002150537634408602,
 0.0017482517482517483,
 0.001388888888888889,
 0.0012594458438287153,
 0.0012547051442910915,
 0.0010298661174047373,
 0.0010080645161290322,
 0.0009871668311944718,
 0.0009718172983479105,
 0.0009633911368015414,
 0.0009578544061302681,
 0.0009478672985781991,
 0.000946073793755913,
 0.0009328358208955224,
 0.0009302325581395349,
 0.0009182736455463728,
 0.0009132420091324201,
 0.0009099181073703367,
 0.0009057971014492754]